In [1]:
from uniem.model import PoolingStrategy, create_uniem_embedder

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from uniem.types import MixedPrecisionType, Tokenizer
import pandas as pd
from typing import Callable, Iterable, Sequence, Sized, cast
from uniem.finetuner import FineTuner
from uniem.training_strategy import BitFitTrainging
from uniem.model import PoolingStrategy, create_uniem_embedder
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
from uniem.utils import *

In [4]:
model_name_or_path = "pretrain_weights/llama2/llama2-7b-hf/"
model_class=AutoModelForCausalLM

In [5]:
pooling_strategy=PoolingStrategy.last_weighted

In [11]:
model = AutoModelForCausalLM.from_pretrained("pretrain_weights/llama2/llama2-7b-hf/")  # type: ignore

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import LlamaForCausalLM

In [ ]:
for n,p in model.named_parameters():
    prefix = "->" if 'bias' in n else ""
    print(f"{prefix}{n}->{p.shape}->{p.max()}-{p.min()}->{p.dtype}")

In [12]:

for n,p in model.named_parameters():
    prefix = "->" if 'bias' in n else ""
    print(f"{prefix}{n}->{p.shape}->{p.max()}-{p.min()}->{p.dtype}")

model.embed_tokens.weight->torch.Size([32000, 4096])->0.26171875--0.13671875->torch.float32
model.layers.0.self_attn.q_proj.weight->torch.Size([4096, 4096])->0.72265625--0.7734375->torch.float32
model.layers.0.self_attn.k_proj.weight->torch.Size([4096, 4096])->0.703125--0.81640625->torch.float32
model.layers.0.self_attn.v_proj.weight->torch.Size([4096, 4096])->0.134765625--0.1240234375->torch.float32
model.layers.0.self_attn.o_proj.weight->torch.Size([4096, 4096])->0.416015625--0.451171875->torch.float32
model.layers.0.mlp.gate_proj.weight->torch.Size([11008, 4096])->0.7265625--0.890625->torch.float32
model.layers.0.mlp.up_proj.weight->torch.Size([11008, 4096])->0.314453125--0.341796875->torch.float32
model.layers.0.mlp.down_proj.weight->torch.Size([4096, 11008])->0.52734375--0.51171875->torch.float32
model.layers.0.input_layernorm.weight->torch.Size([4096])->0.80078125--0.00092315673828125->torch.float32
model.layers.0.post_attention_layernorm.weight->torch.Size([4096])->0.2197265625-

model.layers.19.mlp.up_proj.weight->torch.Size([11008, 4096])->0.486328125--0.5703125->torch.float32
model.layers.19.mlp.down_proj.weight->torch.Size([4096, 11008])->0.68359375--0.96875->torch.float32
model.layers.19.input_layernorm.weight->torch.Size([4096])->0.8515625-5.2928924560546875e-05->torch.float32
model.layers.19.post_attention_layernorm.weight->torch.Size([4096])->0.369140625-0.06591796875->torch.float32
model.layers.20.self_attn.q_proj.weight->torch.Size([4096, 4096])->0.8828125--1.0546875->torch.float32
model.layers.20.self_attn.k_proj.weight->torch.Size([4096, 4096])->0.328125--0.328125->torch.float32
model.layers.20.self_attn.v_proj.weight->torch.Size([4096, 4096])->0.126953125--0.1630859375->torch.float32
model.layers.20.self_attn.o_proj.weight->torch.Size([4096, 4096])->0.57421875--0.578125->torch.float32
model.layers.20.mlp.gate_proj.weight->torch.Size([11008, 4096])->0.287109375--0.322265625->torch.float32
model.layers.20.mlp.up_proj.weight->torch.Size([11008, 4096])

model.layers.30.self_attn.q_proj.weight->torch.Size([4096, 4096])->0.81640625--0.75->torch.float32
model.layers.30.self_attn.k_proj.weight->torch.Size([4096, 4096])->0.369140625--0.34375->torch.float32
model.layers.30.self_attn.v_proj.weight->torch.Size([4096, 4096])->0.5--0.43359375->torch.float32
model.layers.30.self_attn.o_proj.weight->torch.Size([4096, 4096])->0.625--0.51953125->torch.float32
model.layers.30.mlp.gate_proj.weight->torch.Size([11008, 4096])->0.7109375--0.796875->torch.float32
model.layers.30.mlp.up_proj.weight->torch.Size([11008, 4096])->0.7109375--0.77734375->torch.float32
model.layers.30.mlp.down_proj.weight->torch.Size([4096, 11008])->1.046875--1.3046875->torch.float32
model.layers.30.input_layernorm.weight->torch.Size([4096])->1.125--8.153915405273438e-05->torch.float32
model.layers.30.post_attention_layernorm.weight->torch.Size([4096])->0.6171875-0.08935546875->torch.float32
model.layers.31.self_attn.q_proj.weight->torch.Size([4096, 4096])->0.41796875--0.6679687